In [1]:
import numpy as np
import stock
import xueqiu
import kline
import status

# 日线崛起态势的股票进行显示

In [2]:
def isRasing(a,c,istoday):
    #istoday True可以使用xueqiu数据
    #c [0 company_id,1 code,2 name,3 category,4 ttm,5 pb]
    #a[0] a[1] a[2]最近三天的数据a[2]是最近的一天
    #macd 在两日日内有翻红趋势，energy从1以下崛起到大于5，volumeJ趋势向上,close趋势向上
    #0 id , 1 close , 2 volume , 3 volumema20 , 4 macd , 5 energy ,6 volumeJ
    dMACD = a[-1][4]-a[-2][4]
    dVolumeJ = a[-1][6]-a[-2][6]
    dClose = a[-1][1]-a[-2][1]
    if istoday:
        #做一些过滤因为太多下载的数据就会太多
        if a[-2][5]<5 and status.isPopularCategory(c[3]): #macd斜率向上，volumeJ斜率向上，能量<5
            b,k,d = xueqiu.xueqiuK15day(c[1])
            if b:
                a.append([0,k[4],k[0],0,0,0,0])
                A = np.array(a)
                energyJ = stock.kdj(stock.volumeEnergy(A[:,2]))[:,2]
                m = stock.macdV(A[:,1])
                if m[-1]-m[-2]>0 and energyJ[-1]>=5: #能量崛起
                    print(c)
                    return True
    else:
        #if a[2][4]<0 and a[2][4] + 2*dMACD >= 0 and dVolumeJ>0 and dClose>0 and a[2][5]>=5 and a[1][5]<5: #预macd两日内翻红，volumeJ斜率向上,close增长,energy崛起
        if dMACD >= 0 and dVolumeJ>0 and dClose>0 and a[-1][5]>=5 and a[-2][5]<5 and status.isPopularCategory(c[3]): #macd斜率向上，volumeJ斜率向上,close增长,energy崛起
            return True
    
    return False
status.RasingCategoryList('d',isRasing)

IntProgress(value=0, description='Loading:', layout=Layout(display='flex', flex_flow='wrap', width='100%'))

Output()

Box(children=(Button(description='2020-01-23', style=ButtonStyle()), Button(description='2020-01-22', style=Bu…

Output()